# Statistik zum Outcome type
Es soll ein Barplot geben, der je nach Verbleibdauer (in Jahren) zeigen soll:

- **Alle positiven **
    + Adoption
    + Return_to_owner
    + Transfer
    
- ** Alle negativen **
    + Euthanesia 
    + Died

In [227]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

%matplotlib inline

In [228]:
data = pd.read_csv("../../data/train.csv", index_col=0)
data.head(2)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


In [261]:
data.AnimalType = data.AnimalType.astype("category")
data.OutcomeSubtype = data.OutcomeSubtype.astype("category")
data.OutcomeType = data.OutcomeType.astype("category")
data.AnimalType.cat.codes.value_counts().sort_index()

0    11134
1    15595
dtype: int64

In [247]:
data.columns

Index(['Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype', 'AnimalType',
       'SexuponOutcome', 'Breed', 'Color', 'AgeuponOutcomeDays'],
      dtype='object')

#### Das Problem hier ist, dass in AgeuponOutcome mit unterschiedlichen Einheiten (Jahr, Jahre, Monat, Monate, Woche, Wochen, ...) gerechnet wird. Also muss das erstmal auf Jahre gerechnet werden.
Es gibt auch noch den blöden Sonderfall "0 years". Ich rechne das mal auf einen Tag um ...

In [248]:
data.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,Breed,Color,AgeuponOutcomeDays
AnimalID,,,,,,,,,
A006100,Scamp,2014-12-20 16:35:00,Return_to_owner,NaN,Dog,Neutered Male,Spinone Italiano Mix,Yellow/White,2555.0
A047759,Oreo,2014-04-07 15:12:00,Transfer,Partner,Dog,Neutered Male,Dachshund,Tricolor,3650.0
A134067,Bandit,2013-11-16 11:54:00,Return_to_owner,NaN,Dog,Neutered Male,Shetland Sheepdog,Brown/White,5840.0
A141142,Bettie,2013-11-17 11:40:00,Return_to_owner,NaN,Dog,Spayed Female,Labrador Retriever/Pit Bull,Black/White,5475.0
A163459,Sasha,2014-11-14 19:28:00,Return_to_owner,NaN,Dog,Intact Female,Miniature Schnauzer Mix,Black/Gray,5475.0


In [232]:
print(data[data.AgeuponOutcome.isnull()].size)
data.AgeuponOutcome = data.AgeuponOutcome.fillna("0 days")
print(data[data.AgeuponOutcome.isnull()].size)

#data[data.Name.isnull()].size
#data.AgeuponOutcome.median()

162
0


In [237]:
data[(~data.AgeuponOutcome.isnull()) & (data.AgeuponOutcome.str.contains("day"))].AgeuponOutcome.value_counts()#.head(5)

AttributeError: 'DataFrame' object has no attribute 'AgeuponOutcome'

In [2]:
s = data.AgeuponOutcome.str.split(" ")

split = DataFrame({"AgeuponOutcomeValue": s.apply(Series, 1)[0], 
                   "AgeuponOutcomeUnit": s.apply(Series, 1)[1]})


data = data.join(split)

data.AgeuponOutcomeUnit = data.AgeuponOutcomeUnit.str.replace("s$", "")
data.AgeuponOutcomeValue = data.AgeuponOutcomeValue.astype(float)

data.AgeuponOutcomeUnit.value_counts()

NameError: name 'data' is not defined

In [1]:
vals = data[data.AgeuponOutcomeUnit == "year"].AgeuponOutcomeValue * 365
vals = vals.append(data[data.AgeuponOutcomeUnit == "month"].AgeuponOutcomeValue * 30)
vals = vals.append(data[data.AgeuponOutcomeUnit == "day"].AgeuponOutcomeValue)
vals.rename("AgeuponOutcomeDays", inplace=True)

vals.sort_index(inplace=True)
data.sort_index(inplace=True)
data = data.join(vals)
data.drop(["AgeuponOutcome", "AgeuponOutcomeUnit", "AgeuponOutcomeValue"], 1, inplace=True)
data.head(1)

NameError: name 'data' is not defined

In [212]:
data.AgeuponOutcomeDays.value_counts()

365.0     3969
730.0     3742
60.0      3397
1095.0    1823
30.0      1281
90.0      1277
1460.0    1071
1825.0     992
120.0      888
2190.0     670
150.0      652
180.0      588
2920.0     536
2555.0     531
300.0      457
3650.0     446
240.0      402
3285.0     288
210.0      288
4380.0     234
270.0      224
330.0      166
4745.0     143
4015.0     126
3.0        109
2.0         99
5110.0      97
5475.0      85
1.0         66
4.0         50
6.0         50
5840.0      36
5.0         24
0.0         22
6205.0      17
6570.0      10
6935.0       3
7300.0       2
Name: AgeuponOutcomeDays, dtype: int64

In [132]:
from sklearn import tree
X = [[0, 0], [2, 2], [5, 6]]
Y = [0, 5, 1]
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
clf.predict([[5., 5.]])

#iris = load_iris()
#clf = tree.DecisionTreeClassifier()
#clf = clf.fit(iris.data, iris.target)

array([1])

In [133]:
data[~data.Color.str.contains("/")].Color.value_counts()

Black                2292
Brown Tabby          1635
White                 931
Orange Tabby          841
Tricolor              752
Brown                 639
Tan                   628
Tortie                530
Calico                517
Blue                  450
Blue Tabby            433
Red                   337
Torbie                335
Brown Brindle         232
Buff                  199
Sable                 198
Cream Tabby           198
Yellow                185
Lynx Point            168
Cream                 151
Chocolate             137
Seal Point            136
Gray                   94
Flame Point            85
Blue Merle             77
Fawn                   73
Gold                   52
Black Smoke            46
Black Tabby            42
Lilac Point            39
Gray Tabby             35
Brown Merle            35
Tortie Point           32
Blue Point             29
Silver Tabby           28
Red Merle              26
Calico Point           25
Apricot                21
Black Brindl

In [134]:
data.AnimalType.value_counts()

Dog    15595
Cat    11134
Name: AnimalType, dtype: int64

In [135]:
data.head()

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,Breed,Color,AgeuponOutcomeWeeks
AnimalID,,,,,,,,,
A006100,Scamp,2014-12-20 16:35:00,Return_to_owner,NaN,Dog,Neutered Male,Spinone Italiano Mix,Yellow/White,364.0
A047759,Oreo,2014-04-07 15:12:00,Transfer,Partner,Dog,Neutered Male,Dachshund,Tricolor,520.0
A134067,Bandit,2013-11-16 11:54:00,Return_to_owner,NaN,Dog,Neutered Male,Shetland Sheepdog,Brown/White,832.0
A141142,Bettie,2013-11-17 11:40:00,Return_to_owner,NaN,Dog,Spayed Female,Labrador Retriever/Pit Bull,Black/White,780.0
A163459,Sasha,2014-11-14 19:28:00,Return_to_owner,NaN,Dog,Intact Female,Miniature Schnauzer Mix,Black/Gray,780.0


In [240]:
md = data[data.AnimalType.isin(["Dog", "Cat"])]
md.AnimalType = md.AnimalType.replace("Dog", "1")
md.AnimalType = md.AnimalType.replace("Cat", "2")

md.Color = md.Color.replace("Black", 1)
md.Color = md.Color.replace("White", 2)
md.Color = md.Color.replace("Pink", 3)

md = md[md.Color.isin([1,2,3])]

md.OutcomeType = md.Color.replace("Died", 1)
md.OutcomeType = md.Color.replace("Adoption", 2)

md = md[md.OutcomeType.isin([1,2])]

md.head(10)

#md.OutcomeType.value_counts()

#md[["AnimalType", "Color"]].values
#md["OutcomeType"].values

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,Breed,Color,AgeuponOutcomeDays
AnimalID,,,,,,,,,
A214991,Prissy,2013-12-14 13:28:00,1,Suffering,2,Spayed Female,Domestic Shorthair,1,6935.0
A215368,Guiness,2014-08-04 16:31:00,1,NaN,1,Neutered Male,Newfoundland Mix,1,4745.0
A275975,Squeakers,2013-10-12 11:27:00,1,NaN,2,Spayed Female,Domestic Shorthair Mix,1,4015.0
A276780,Seth,2013-11-22 15:56:00,1,NaN,1,Neutered Male,Border Collie/Chow Chow,1,4015.0
A287217,Spider,2014-10-31 13:06:00,1,NaN,2,Spayed Female,Domestic Shorthair Mix,1,4380.0
A295822,Fuzzy,2015-05-09 18:47:00,1,NaN,2,Spayed Female,Domestic Longhair Mix,1,4380.0
A306362,Zoey,2015-07-18 16:40:00,1,NaN,1,Spayed Female,Labrador Retriever/Chow Chow,1,4745.0
A341599,Reece,2014-01-11 11:05:00,1,Partner,2,Neutered Male,Domestic Shorthair Mix,1,3285.0
A346599,Assassin,2015-12-04 12:22:00,1,Partner,2,Intact Male,Domestic Shorthair Mix,1,4015.0


In [143]:
#md[["AnimalType", "Color"]].values
md["OutcomeType"].values

array([1, 1, 1, ..., 2, 2, 1])

In [241]:
X = md[["AnimalType", "Color"]].values
Y = md["OutcomeType"].values
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
clf.predict_proba ([[2, 3]])

array([[ 0.,  1.]])